In [ ]:
# Beam Geometry
length = 10.0      # Beam length (m)
width = 350       # Beam width (mm)
height = 600      # Beam height (mm)
cover = 50        # Concrete cover (mm)
effective_depth = height - cover  # Effective depth (mm)

# Loading
dead_load = 30.0  # Dead load (kN/m)
live_load = 35.0  # Live load (kN/m)

# Material Properties
fc = 25           # Concrete compressive strength (MPa)
fy = 420          # Steel yield strength (MPa)

print(f"📐 Beam Dimensions: {width} × {height} mm")
print(f"📏 Span Length: {length} m")
print(f"📦 Dead Load: {dead_load} kN/m")
print(f"🏃 Live Load: {live_load} kN/m")
print(f"🧱 f'c = {fc} MPa")
print(f"🔗 fy = {fy} MPa")


In [ ]:
# Load factors per ACI 318-19
load_factor_dead = 1.2
load_factor_live = 1.6

# Factored loads
factored_load = load_factor_dead * dead_load + load_factor_live * live_load

print(f"⚡ Load Combination: 1.2D + 1.6L")
print(f"🔢 wu = {load_factor_dead} × {dead_load} + {load_factor_live} × {live_load}")
print(f"📊 Factored Load: wu = {factored_load:.1f} kN/m")


In [ ]:
import numpy as np

# Maximum moment and shear for simply supported beam
max_moment = factored_load * length**2 / 8  # kN·m
max_shear = factored_load * length / 2      # kN

# Convert to N·mm for design
mu = max_moment * 1e6  # N·mm
vu = max_shear * 1000  # N

print(f"💪 Maximum Moment: Mu = {max_moment:.1f} kN·m")
print(f"✂️  Maximum Shear: Vu = {max_shear:.1f} kN")
print(f"🔧 Design Moment: Mu = {mu:.0f} N·mm")
print(f"🔧 Design Shear: Vu = {vu:.0f} N")


In [ ]:
# Material strength reduction factors (ACI 318-19)
phi_flexure = 0.9

# Concrete stress parameters
beta1 = 0.85 if fc <= 28 else max(0.65, 0.85 - 0.05 * (fc - 28) / 7)
print(f"📏 β₁ = {beta1:.3f}")

# Required steel area calculation
# Using approximate method: As = Mu / (phi * fy * 0.9 * d)
arm_approx = 0.9 * effective_depth  # Approximate lever arm
steel_area_req = mu / (phi_flexure * fy * arm_approx)  # mm²

print(f"🎯 Required Steel Area: As,req = {steel_area_req:.0f} mm²")
print(f"📊 Steel Ratio: ρ = {steel_area_req / (width * effective_depth) * 100:.3f}%")


In [ ]:
# Standard bar areas (mm²)
bar_areas = {
    12: 113, 16: 201, 20: 314, 25: 491, 32: 804, 40: 1257
}

# Select reinforcement
bar_diameter = 25  # mm
bar_area = bar_areas[bar_diameter]
num_bars = int(np.ceil(steel_area_req / bar_area))
steel_area_provided = num_bars * bar_area

print(f"🔗 Selected: {num_bars} × Ø{bar_diameter}mm bars")
print(f"📦 Provided Steel Area: As = {steel_area_provided} mm²")
print(f"✅ Area Check: {steel_area_provided / steel_area_req:.2f} × required")

# Check minimum steel
as_min = max(0.25 * np.sqrt(fc) / fy * width * effective_depth,
             1.4 / fy * width * effective_depth)
print(f"⚠️  Minimum Steel: As,min = {as_min:.0f} mm²")
print(f"✅ Min Check: {'OK' if steel_area_provided >= as_min else 'FAIL'}")


In [ ]:
# Shear design per ACI 318-19
phi_shear = 0.75

# Concrete shear capacity
vc = 0.17 * np.sqrt(fc) * width * effective_depth  # N
phi_vc = phi_shear * vc

print(f"🧱 Concrete Shear Capacity: φVc = {phi_vc / 1000:.1f} kN")
print(f"⚡ Applied Shear: Vu = {vu / 1000:.1f} kN")

if vu <= phi_vc:
    print(f"✅ Shear: OK - Concrete capacity sufficient")
    stirrup_requirement = "Minimum stirrups per ACI 318-19"
else:
    vs_required = vu - phi_vc  # Required steel shear capacity
    print(f"⚠️  Shear reinforcement required: Vs = {vs_required / 1000:.1f} kN")
    stirrup_requirement = f"Design stirrups for Vs = {vs_required / 1000:.1f} kN"

print(f"🔗 Stirrups: {stirrup_requirement}")


In [ ]:
print("\n" + "="*50)
print("🏗️  GHALI CONSULTANTS - DESIGN SUMMARY")
print("="*50)
print(f"Beam: {width} × {height} mm, Span = {length} m")
print(f"Loading: DL = {dead_load} kN/m, LL = {live_load} kN/m")
print(f"Design Forces: Mu = {max_moment:.1f} kN·m, Vu = {max_shear:.1f} kN")
print(f"Main Steel: {num_bars} × Ø{bar_diameter}mm (As = {steel_area_provided} mm²)")
print(f"Stirrups: {stirrup_requirement}")
print(f"Materials: f'c = {fc} MPa, fy = {fy} MPa")
print("="*50)
print("✅ Design completed per ACI 318-19")
print("📄 Ready for PDF generation")
